In [1]:
import pandas as pd
import numpy as np
import plotly as py
from plotly import tools
import plotly.graph_objs as go
from feature_functions import *
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from scipy import stats
import scipy.optimize
from scipy.optimize import OptimizeWarning
import warnings
from sklearn.linear_model import LinearRegression

# Variables:
future = 15
averages = [5,10,15,20,30,50,70,100,200,300]


# Loading the data
df = pd.read_csv("data/EURUSDhour.csv")
df.columns = ['date','open','high','low','close','volume']
df.date = pd.to_datetime(df.date, format='%d.%m.%Y %H:%M:%S.%f')
df = df.set_index(df.date)
df = df[['open','high','low','close','volume']]

# Drop elements where there is no mouvement (Market closed) like in the weekends
df = df.drop_duplicates(keep=False)
df.shape

(9801, 5)

In [2]:
sine(df, [5,6])

,sine 5 a0,sine 5 b1,sine 5 w,sine 6 a0,sine 6 b1,sine 6 w
date,,,,,,
2017-01-01 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-01 23:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-02 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-02 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-02 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2017-01-02 03:00:00,0.000537,-0.000904,0.659282,NaN,NaN,NaN
2017-01-02 04:00:00,-0.000144,-0.006008,0.002829,0.000402,-0.000700,0.460641
2017-01-02 05:00:00,-0.000388,0.000312,0.793718,-0.000127,-0.025546,0.001174
2017-01-02 06:00:00,-0.000040,-0.000212,0.811354,-0.000193,-0.000230,2.248793


In [ ]:
# The data
interval = 20
run_length = 100
correct_predictions = 0
for i in range(run_length):
    X = df_with_averages.iloc[:df_with_averages.shape[0]- (i*interval),:-1]
    y = df_with_averages.result

    # Split the data
    X_train= X.iloc[:X.shape[0]-1,:]
    X_test= X.iloc[X.shape[0]-1:X.shape[0],:]
    y_train= y[:X.shape[0]-1]
    y_test =y [X.shape[0]-1:X.shape[0]]

    clf = GradientBoostingClassifier(random_state=5, learning_rate=0.01, n_estimators=10000)
    clf.fit(X_train, y_train) 

    # Predictions
    predicted = clf.predict(X_test)
    print (str(accuracy_score(y_test, predicted)))
    correct_predictions += accuracy_score(y_test, predicted)